## Zaraar Malik
## i212705@nu.edu.pk

## IMPORTING NECESSARY LIBRARIES

In [1]:
import random
import spacy
import pandas as pd
import numpy as np

## READING CSV FILE WITH SCRAPPED POEMS

In [2]:
data=pd.read_csv('scrapped_poems.csv',usecols= ['poem_line','nazm_name','author_name'])

## DATAFRAME

In [3]:
data.head()

,poem_line,nazm_name,author_name
0,رگوں میں دوڑتا پھرتا لہو پھر تھم گیا ہے,برأت,عابد ادیب
1,ہوائیں تیز ہیں سانسوں کی ہلچل رک گئی ہے,برأت,عابد ادیب
2,ریڑھ کی ہڈی میں چیونٹی رینگتی ہے,برأت,عابد ادیب
3,جسم میں پورے حرارت بڑھ گئی ہے,برأت,عابد ادیب
4,ذائقہ کڑوا کسیلا ہو گیا ہے,برأت,عابد ادیب


## DATAFRAME SIZE

In [4]:
data.shape

(1856, 3)

## TOTAL NUMBER OF AUTHORS

In [5]:
len(data['author_name'].unique())

30

## NECESSARY FUNCTIONS 

In [63]:
def tokenize_poem_lines(dataframe_series):
    poem_lines_tokenized=[]
    spacy_tokenizer = spacy.blank('ur')
    for i in dataframe_series:
        poem_lines_tokenized.append(list(spacy_tokenizer(i)))
    return poem_lines_tokenized

def Generate_Bigrams(dataframe_series):
    spacy_tokenizer = spacy.blank('ur')
    Bigrams=[]
    for i in dataframe_series:
        holder=[spacy_tokenizer(i[j].text+' '+i[j+1].text) for j in range(len(i)-1)]
        Bigrams.append(holder)
    return Bigrams

def Generate_Trigrams(dataframe_series):
    spacy_tokenizer = spacy.blank('ur')
    Trigrams=[]
    for i in dataframe_series:
        holder=[spacy_tokenizer(i[j].text+' '+i[j+1].text+' '+i[j+2].text) for j in range(len(i)-2)]
        Trigrams.append(holder)
    return Trigrams

def Remove_StopWords(dataframe_series,bigram_flag=False,trigram_flag=False):
    Removed_Stopwords=[]
    if bigram_flag==False and trigram_flag==False:
        for i in dataframe_series:
            filtered_tokens = [token.text for token in i if not token.is_stop]
            Removed_Stopwords.append(filtered_tokens)
        return Removed_Stopwords
    elif bigram_flag==True:
        for i in dataframe_series:
            filtered_tokens =[token.text for token in i if not token[0].is_stop or token[1].is_stop]
            Removed_Stopwords.append(filtered_tokens)
        return Removed_Stopwords
    elif trigram_flag==True:
        for i in dataframe_series:
            filtered_tokens =[token.text for token in i if not token[0].is_stop or token[2].is_stop]
            Removed_Stopwords.append(filtered_tokens)
        return Removed_Stopwords

def get_starting_words(dataframe,selected_poet,ngram):
    starting_words=[]
    temp=dataframe[dataframe['author_name']==selected_poet]
    if ngram=='Unigrams':
        starting_words=[i[0] for i in temp['Updated_Unigrams']]
        return starting_words
    elif ngram=='Bigrams':
        starting_words=[i[0].split()[0] for i in temp['Updated_Bigrams']]
        return starting_words
    elif ngram=='Backward-Bigrams':
        starting_words=[i[0].split()[1] for i in temp['Updated_Bigrams']]
        return starting_words
    elif ngram=='Trigrams':
        starting_words=[i[0].split()[0] for i in temp['Updated_Trigrams']]
        return starting_words

def get_center_words(dataframe,selected_poet):
    starting_words=[]
    temp=dataframe[dataframe['author_name']==selected_poet]
    for i in temp['Updated_Unigrams']:
        if len(i)%2!=0:
            starting_words.append(i[len(i)-1-int(len(i)/2)])
        else:
            starting_words.append(i[int(len(i)/2)])
            starting_words.append(i[int(len(i)/2)-1])
    return starting_words

def Generate_CFD_Unigram(dataframe,selected_poet):
    FD_dict={}
    CFD_dict={}
    UG=dataframe[dataframe['author_name']==selected_poet]['Updated_Unigrams']
    for i in UG:
        for j in i:
            if j not in FD_dict.keys():
                FD_dict[j]=1
            elif j in FD_dict.keys():
                FD_dict[j]=FD_dict[j]+1
    max_value=sum(FD_dict.values())
    for i in FD_dict.keys():
        CFD_dict[i]=np.round(FD_dict[i]/max_value,9)
    return CFD_dict

def Generate_CFD_Bigram(dataframe,selected_poet):
    FD_dict={}
    CFD_dict={}
    UG=dataframe[dataframe['author_name']==selected_poet]['Updated_Bigrams']
    for i in UG:
        for j in i:
            if j not in FD_dict.keys():
                FD_dict[j]=1
            elif j in FD_dict.keys():
                FD_dict[j]=FD_dict[j]+1
    max_value=sum(FD_dict.values())
    for i in FD_dict.keys():
        CFD_dict[i]=np.round(FD_dict[i]/max_value,9)
    return CFD_dict

def Generate_CFD_Trigram(dataframe,selected_poet):
    FD_dict={}
    CFD_dict={}
    UG=dataframe[dataframe['author_name']==selected_poet]['Updated_Trigrams']
    for i in UG:
        for j in i:
            if j not in FD_dict.keys():
                FD_dict[j]=1
            elif j in FD_dict.keys():
                FD_dict[j]=FD_dict[j]+1
    max_value=sum(FD_dict.values())
    for i in FD_dict.keys():
        CFD_dict[i]=np.round(FD_dict[i]/max_value,9)
    return CFD_dict


# DATA PREPARATION #

### TOKENIZING LINES OF THE EXTRACTED POEMS

In [7]:
Poem_Lines_Tokenized=tokenize_poem_lines(data['poem_line'])
data['tokenized_Lines']=Poem_Lines_Tokenized

In [8]:
data.head()

,poem_line,nazm_name,author_name,tokenized_Lines
0,رگوں میں دوڑتا پھرتا لہو پھر تھم گیا ہے,برأت,عابد ادیب,"[رگوں, میں, دوڑتا, پھرتا, لہو, پھر, تھم, گیا, ہے]"
1,ہوائیں تیز ہیں سانسوں کی ہلچل رک گئی ہے,برأت,عابد ادیب,"[ہوائیں, تیز, ہیں, سانسوں, کی, ہلچل, رک, گئی, ہے]"
2,ریڑھ کی ہڈی میں چیونٹی رینگتی ہے,برأت,عابد ادیب,"[ریڑھ, کی, ہڈی, میں, چیونٹی, رینگتی, ہے]"
3,جسم میں پورے حرارت بڑھ گئی ہے,برأت,عابد ادیب,"[جسم, میں, پورے, حرارت, بڑھ, گئی, ہے]"
4,ذائقہ کڑوا کسیلا ہو گیا ہے,برأت,عابد ادیب,"[ذائقہ, کڑوا, کسیلا, ہو, گیا, ہے]"


### GENERATING UNIGRAMS OF THE POEM LINES
#### Unigrams are basically the splitted poem lines so no need to do anything for unigrams

In [9]:
data['Poem_Line_Unigrams']=data['tokenized_Lines']

### GENERATING BIGRAMS OF THE POEM LINES

In [10]:
Poem_Line_Bigrams=Generate_Bigrams(data['tokenized_Lines'])
data['Poem_Line_Bigrams']=Poem_Line_Bigrams

In [11]:
data.head()

,poem_line,nazm_name,author_name,tokenized_Lines,Poem_Line_Unigrams,Poem_Line_Bigrams
0,رگوں میں دوڑتا پھرتا لہو پھر تھم گیا ہے,برأت,عابد ادیب,"[رگوں, میں, دوڑتا, پھرتا, لہو, پھر, تھم, گیا, ہے]","[رگوں, میں, دوڑتا, پھرتا, لہو, پھر, تھم, گیا, ہے]","[(رگوں, میں), (میں, دوڑتا), (دوڑتا, پھرتا), (پ..."
1,ہوائیں تیز ہیں سانسوں کی ہلچل رک گئی ہے,برأت,عابد ادیب,"[ہوائیں, تیز, ہیں, سانسوں, کی, ہلچل, رک, گئی, ہے]","[ہوائیں, تیز, ہیں, سانسوں, کی, ہلچل, رک, گئی, ہے]","[(ہوائیں, تیز), (تیز, ہیں), (ہیں, سانسوں), (سا..."
2,ریڑھ کی ہڈی میں چیونٹی رینگتی ہے,برأت,عابد ادیب,"[ریڑھ, کی, ہڈی, میں, چیونٹی, رینگتی, ہے]","[ریڑھ, کی, ہڈی, میں, چیونٹی, رینگتی, ہے]","[(ریڑھ, کی), (کی, ہڈی), (ہڈی, میں), (میں, چیون..."
3,جسم میں پورے حرارت بڑھ گئی ہے,برأت,عابد ادیب,"[جسم, میں, پورے, حرارت, بڑھ, گئی, ہے]","[جسم, میں, پورے, حرارت, بڑھ, گئی, ہے]","[(جسم, میں), (میں, پورے), (پورے, حرارت), (حرار..."
4,ذائقہ کڑوا کسیلا ہو گیا ہے,برأت,عابد ادیب,"[ذائقہ, کڑوا, کسیلا, ہو, گیا, ہے]","[ذائقہ, کڑوا, کسیلا, ہو, گیا, ہے]","[(ذائقہ, کڑوا), (کڑوا, کسیلا), (کسیلا, ہو), (ہ..."


### GENERATING TRIGRAMS OF THE POEM LINES

In [12]:
Poem_Line_Trigrams=Generate_Trigrams(data['tokenized_Lines'])
data['Poem_Line_Trigrams']=Poem_Line_Trigrams

In [13]:
data.head()

,poem_line,nazm_name,author_name,tokenized_Lines,Poem_Line_Unigrams,Poem_Line_Bigrams,Poem_Line_Trigrams
0,رگوں میں دوڑتا پھرتا لہو پھر تھم گیا ہے,برأت,عابد ادیب,"[رگوں, میں, دوڑتا, پھرتا, لہو, پھر, تھم, گیا, ہے]","[رگوں, میں, دوڑتا, پھرتا, لہو, پھر, تھم, گیا, ہے]","[(رگوں, میں), (میں, دوڑتا), (دوڑتا, پھرتا), (پ...","[(رگوں, میں, دوڑتا), (میں, دوڑتا, پھرتا), (دوڑ..."
1,ہوائیں تیز ہیں سانسوں کی ہلچل رک گئی ہے,برأت,عابد ادیب,"[ہوائیں, تیز, ہیں, سانسوں, کی, ہلچل, رک, گئی, ہے]","[ہوائیں, تیز, ہیں, سانسوں, کی, ہلچل, رک, گئی, ہے]","[(ہوائیں, تیز), (تیز, ہیں), (ہیں, سانسوں), (سا...","[(ہوائیں, تیز, ہیں), (تیز, ہیں, سانسوں), (ہیں,..."
2,ریڑھ کی ہڈی میں چیونٹی رینگتی ہے,برأت,عابد ادیب,"[ریڑھ, کی, ہڈی, میں, چیونٹی, رینگتی, ہے]","[ریڑھ, کی, ہڈی, میں, چیونٹی, رینگتی, ہے]","[(ریڑھ, کی), (کی, ہڈی), (ہڈی, میں), (میں, چیون...","[(ریڑھ, کی, ہڈی), (کی, ہڈی, میں), (ہڈی, میں, چ..."
3,جسم میں پورے حرارت بڑھ گئی ہے,برأت,عابد ادیب,"[جسم, میں, پورے, حرارت, بڑھ, گئی, ہے]","[جسم, میں, پورے, حرارت, بڑھ, گئی, ہے]","[(جسم, میں), (میں, پورے), (پورے, حرارت), (حرار...","[(جسم, میں, پورے), (میں, پورے, حرارت), (پورے, ..."
4,ذائقہ کڑوا کسیلا ہو گیا ہے,برأت,عابد ادیب,"[ذائقہ, کڑوا, کسیلا, ہو, گیا, ہے]","[ذائقہ, کڑوا, کسیلا, ہو, گیا, ہے]","[(ذائقہ, کڑوا), (کڑوا, کسیلا), (کسیلا, ہو), (ہ...","[(ذائقہ, کڑوا, کسیلا), (کڑوا, کسیلا, ہو), (کسی..."


## Removing Stopwords 

In [14]:
Unigrams_Removed_Stopwords=Remove_StopWords(data['Poem_Line_Unigrams'])
Bigrams_Removed_Stopwords=Remove_StopWords(data['Poem_Line_Bigrams'],True,False)
Trigrams_Removed_Stopwords=Remove_StopWords(data['Poem_Line_Trigrams'],False,True)

data['Updated_Unigrams']=Unigrams_Removed_Stopwords
data['Updated_Bigrams']=Bigrams_Removed_Stopwords
data['Updated_Trigrams']=Trigrams_Removed_Stopwords

In [15]:
data.head()

,poem_line,nazm_name,author_name,tokenized_Lines,Poem_Line_Unigrams,Poem_Line_Bigrams,Poem_Line_Trigrams,Updated_Unigrams,Updated_Bigrams,Updated_Trigrams
0,رگوں میں دوڑتا پھرتا لہو پھر تھم گیا ہے,برأت,عابد ادیب,"[رگوں, میں, دوڑتا, پھرتا, لہو, پھر, تھم, گیا, ہے]","[رگوں, میں, دوڑتا, پھرتا, لہو, پھر, تھم, گیا, ہے]","[(رگوں, میں), (میں, دوڑتا), (دوڑتا, پھرتا), (پ...","[(رگوں, میں, دوڑتا), (میں, دوڑتا, پھرتا), (دوڑ...","[رگوں, میں, دوڑتا, پھرتا, لہو, تھم, گیا]","[رگوں میں, میں دوڑتا, دوڑتا پھرتا, پھرتا لہو, ...","[رگوں میں دوڑتا, میں دوڑتا پھرتا, دوڑتا پھرتا ..."
1,ہوائیں تیز ہیں سانسوں کی ہلچل رک گئی ہے,برأت,عابد ادیب,"[ہوائیں, تیز, ہیں, سانسوں, کی, ہلچل, رک, گئی, ہے]","[ہوائیں, تیز, ہیں, سانسوں, کی, ہلچل, رک, گئی, ہے]","[(ہوائیں, تیز), (تیز, ہیں), (ہیں, سانسوں), (سا...","[(ہوائیں, تیز, ہیں), (تیز, ہیں, سانسوں), (ہیں,...","[ہوائیں, تیز, سانسوں, ہلچل, رک]","[ہوائیں تیز, تیز ہیں, سانسوں کی, ہلچل رک, رک گ...","[ہوائیں تیز ہیں, تیز ہیں سانسوں, ہیں سانسوں کی..."
2,ریڑھ کی ہڈی میں چیونٹی رینگتی ہے,برأت,عابد ادیب,"[ریڑھ, کی, ہڈی, میں, چیونٹی, رینگتی, ہے]","[ریڑھ, کی, ہڈی, میں, چیونٹی, رینگتی, ہے]","[(ریڑھ, کی), (کی, ہڈی), (ہڈی, میں), (میں, چیون...","[(ریڑھ, کی, ہڈی), (کی, ہڈی, میں), (ہڈی, میں, چ...","[ریڑھ, ہڈی, میں, چیونٹی, رینگتی]","[ریڑھ کی, ہڈی میں, میں چیونٹی, چیونٹی رینگتی, ...","[ریڑھ کی ہڈی, ہڈی میں چیونٹی, میں چیونٹی رینگت..."
3,جسم میں پورے حرارت بڑھ گئی ہے,برأت,عابد ادیب,"[جسم, میں, پورے, حرارت, بڑھ, گئی, ہے]","[جسم, میں, پورے, حرارت, بڑھ, گئی, ہے]","[(جسم, میں), (میں, پورے), (پورے, حرارت), (حرار...","[(جسم, میں, پورے), (میں, پورے, حرارت), (پورے, ...","[جسم, میں, پورے, حرارت, بڑھ]","[جسم میں, میں پورے, پورے حرارت, حرارت بڑھ, بڑھ...","[جسم میں پورے, میں پورے حرارت, پورے حرارت بڑھ,..."
4,ذائقہ کڑوا کسیلا ہو گیا ہے,برأت,عابد ادیب,"[ذائقہ, کڑوا, کسیلا, ہو, گیا, ہے]","[ذائقہ, کڑوا, کسیلا, ہو, گیا, ہے]","[(ذائقہ, کڑوا), (کڑوا, کسیلا), (کسیلا, ہو), (ہ...","[(ذائقہ, کڑوا, کسیلا), (کڑوا, کسیلا, ہو), (کسی...","[ذائقہ, کڑوا, کسیلا, گیا]","[ذائقہ کڑوا, کڑوا کسیلا, کسیلا ہو, گیا ہے]","[ذائقہ کڑوا کسیلا, کڑوا کسیلا ہو, کسیلا ہو گیا..."


# STANDARD N-GRAM MODELS

## UNIGRAM MODEL

In [16]:
def Unigram_Model(dataframe,selected_poet,SL,VMIN,VMAX,VPS):
    a=Generate_CFD_Unigram(dataframe,selected_poet)
    a=sorted(a.items(), key=lambda x:x[1],reverse=True)
    starting_words=get_starting_words(dataframe,selected_poet,'Unigrams')
    for i in range(SL):
        for j in range(VPS):
            random_number=np.random.randint(VMIN,VMAX)
            first_word=random.sample(starting_words,1)
            verse=[a[v][0] for v in range(1,random_number) ]
            print(first_word[0],' '.join(verse))
        print('\n')

## BIGRAM MODEL

In [17]:
def Bigram_Model(dataframe,selected_poet,SL,VMIN,VMAX,VPS):
    b=Generate_CFD_Bigram(dataframe,selected_poet)
    b=sorted(b.items(), key=lambda x:x[1],reverse=True)
    starting_words=get_starting_words(dataframe,selected_poet,'Bigrams')
    first_word=random.sample(starting_words,1)[0]
    stanzas=[]
    for i in range(SL):
        for j in range(VPS):
            verse=''+first_word
            random_number=np.random.randint(VMIN,VMAX)
            for i in range(random_number):
                next_value=''
                for v in b:
                    if v[0].split()[0]==first_word:
                        next_value=v[0]
                if next_value=='':
                    next_value=random.sample(b,1)[0][0]
                first_word=next_value.split()[1]
                verse=verse+' '+first_word
            stanzas.append(verse)
            verse=''
            first_word=random.sample(starting_words,1)[0]
        stanzas.append('\n')
    return stanzas

## TRIGRAM MODEL

In [18]:
def Trigram_Model(dataframe,selected_poet,SL,VMIN,VMAX,VPS):
    b=Generate_CFD_Trigram(dataframe,selected_poet)
    b=sorted(b.items(), key=lambda x:x[1],reverse=True)
    starting_words=get_starting_words(dataframe,selected_poet,'Trigrams')
    first_word=random.sample(starting_words,2)
    stanzas=[]
    for i in range(SL):
        for j in range(VPS):
            verse=''+first_word[0]+' '+first_word[1]
            random_number=np.random.randint(VMIN,VMAX)
            for i in range(random_number):
                next_value=''
                for v in b:
                    if v[0].split()[0]==first_word[0] and v[0].split()[1]==first_word[1]:
                        next_value=v[0]
                        break
                if next_value=='':
                    next_value=random.sample(b,1)[0][0]
                verse=verse+' '+next_value.split()[2]
                first_word=[next_value.split()[1],next_value.split()[2]]
                
            stanzas.append(verse)
            verse=''
            first_word=random.sample(starting_words,2)
        stanzas.append('\n')
    return stanzas

In [19]:
stanza_length=4
verse_min_length=7
verse_max_length=11
verse_per_stanza=4

## UNIGRAM  RESULT

In [20]:
Unigram_Model(data,'ابر احسنی گنوری',stanza_length,verse_min_length,verse_max_length,verse_per_stanza)

گھنے نہ دیا کیا نے میرے سے وطن کو
جو نہ دیا کیا نے میرے سے وطن
زندگی نہ دیا کیا نے میرے سے وطن کو کس
چاند نہ دیا کیا نے میرے سے وطن کو


میرے نہ دیا کیا نے میرے سے
سبزے نہ دیا کیا نے میرے سے وطن کو کس
گھنے نہ دیا کیا نے میرے سے وطن
کیا نہ دیا کیا نے میرے سے وطن


پتے نہ دیا کیا نے میرے سے وطن کو
کس نہ دیا کیا نے میرے سے
جوتیں نہ دیا کیا نے میرے سے
آپس نہ دیا کیا نے میرے سے وطن


جو نہ دیا کیا نے میرے سے وطن
پودے نہ دیا کیا نے میرے سے وطن کو کس
دھانوں نہ دیا کیا نے میرے سے
کس نہ دیا کیا نے میرے سے وطن




## BIGRAM RESULT

In [21]:
temp=Bigram_Model(data,'ابر احسنی گنوری',stanza_length,verse_min_length,verse_max_length,verse_per_stanza)
for i in temp:
    print(i)

کیا یہاں نہیں ہے لوگ کا بیر چھوڑیں مل جل کے
کس لئے کو ساری دنیا سنبھل رہی ہے لوگ
نے بے شمار دیا تو رہی ہے لوگ
میرے لیے تیرا گن گاؤں میں اس زمیں کی


چاہیے سر تجھے جھکاؤں میں اس زمیں کی
سوتا ہوا کا بیر چھوڑیں مل جل کے
پتے ہوا تیری کیا یہاں نہیں ہے لوگ کی یہ
دھانوں کی یہ ہی طرح کی یہ نہ کیوں


دنیا سنبھل رہی ہے لوگ ہے لوگ ہی
کس لئے علم سے کیا یہاں نہیں ہے لوگ جڑے
کیا یہاں نہیں ہے لوگ جن کو ساری دنیا سنبھل رہی
ایسی زمین پا کر ایک لئے جو میں اس زمیں کی


سمجھ دی کہ اک خزانہ اس زمیں کی یہ رہی
دنیا سنبھل رہی ہے لوگ کیوں پہاڑ سے کیا
میرے لیے تیرا گن گاؤں میں اس زمیں کی یہ یہی
آپس کا بیر چھوڑیں مل جل کے ہی مجھے علم سے




## TRIGRAM RESULT

In [22]:
temp=Trigram_Model(data,'ابر احسنی گنوری',stanza_length,5,9,verse_per_stanza)
for i in temp:
    print(i)

مکا کس رہی ہے طرح کی لذت ملک کی
سمندر یہ سناتے تن کر کے عیش یا بکری کروں
کھیتیاں اونٹ ہے یاقوت مجھے سواری دیں ہے کو بھول
گرمی مجھ کر بڑھائیں زمیں سے دولت ابل رہی


بڑھ کیا کے عیش بتاؤں میں یہ
پکھراج میرے ہوئے ہیں دولت ابل رہی ہے یہی
میرے اونٹ میں میں نہ جو جگمگاتے
کس پھر خوشیاں کیے پیدا گاؤں میں ہے ہوئی ہے


جو لازم لوگ تو نے گن نہ
کیا پاتا میری کھیتی ساری مخلوق میں
کتنا کیا طرح کی لذت سنبھل رہی
افلاس میرے جڑے ہوئے ہیں رہی ہے یہ


کیا سوتا یہی میری کھیتی کی لذت رہی ہے
گرمی سبزے مقدر میں ان کی زمیں ہے
کیا دین یہ دیا ہے کو بھول جاؤں میں
دودھ میرے لیے کیے پیدا سے سنوار دیا




# BACKWARD BIGRAM MODEL

In [23]:
def Backward_Bigram_Model(dataframe,selected_poet,SL,VMIN,VMAX,VPS):
    b=Generate_CFD_Bigram(dataframe,selected_poet)
    b=sorted(b.items(), key=lambda x:x[1],reverse=True)
    starting_words=get_starting_words(dataframe,selected_poet,'Backward-Bigrams')
    first_word=random.sample(starting_words,1)[0]
    stanzas=[]
    for i in range(SL):
        for j in range(VPS):
            verse=''+first_word
            random_number=np.random.randint(VMIN,VMAX)
            for i in range(random_number):
                next_value=''
                for v in b:
                    if v[0].split()[1]==first_word:
                        next_value=v[0]
                if next_value=='':
                    next_value=random.sample(b,1)[0][0]
                first_word=next_value.split()[0]
                verse=verse+' '+first_word
            stanzas.append(verse)
            verse=''
            first_word=random.sample(starting_words,1)[0]
        stanzas.append('\n')
    return stanzas

In [24]:
temp=Backward_Bigram_Model(data,'ابر احسنی گنوری',stanza_length,verse_min_length,verse_max_length,verse_per_stanza)
for i in temp:
    print(i)

وطن اپنی سمجھوں نہ خوشیاں چنے میں دنیا سنبھلیں موتی یا
گھی دودھ تیرا لیے میرے نے چھپی میں دنیا
ہے لازم جاؤں بھول احسان ہوئے کھڑے نہ
پھول پھل رستہ سیدھا دنیا سنبھلیں پیار کا


رستہ سیدھا کو جن کو جن کو جن کو جن کو
وطن اپنی سمجھوں نہ خوشیاں یا بیل بھینس
زمین ایسی پکاتے یہی بوئیں جوتیں بیل بھینس یا بیل بھینس
اور گھنے نے دیں سواری مجھے دیں سواری مجھے دیں


کو جن کو جن کو جن کو جن کو جن کو
دی تندرستی سے کیا سے کیا سے کیا
اور گھنے لہلہاتے کھیت لہلہاتے کھیت لہلہاتے کھیت لہلہاتے کھیت
بھی یاقوت کو جن کو جن کو جن کو جن


کو جن کو جن کو جن کو جن کو
نہیں یہاں کیا سے کیا سے کیا سے کیا سے کیا
ہے لازم وطن اپنی سمجھوں نہ خوشیاں گن تیرا لیے میرے
ماتا نے لیے میرے نے پیار کا آپس




# BIDIRECTIONAL BIGRAM MODEL

In [79]:
def right_side_Model(dataframe,selected_poet,VMIN,VMAX,word):
    b=Generate_CFD_Bigram(dataframe,selected_poet)
    b=sorted(b.items(), key=lambda x:x[1],reverse=True)
    first_word=word
    stanzas=[]
    verse=''+first_word
    random_number=np.random.randint(VMIN,VMAX)
    for i in range(random_number):
        next_value=''
        for v in b:
            if v[0].split()[0]==first_word:
                next_value=v[0]
        if next_value=='':
            next_value=random.sample(b,1)[0][0]
        first_word=next_value.split()[1]
        verse=verse+' '+first_word
    stanzas.append(verse)
    return stanzas

In [88]:
def left_side_Model(dataframe,selected_poet,VMIN,VMAX,word):
    b=Generate_CFD_Bigram(dataframe,selected_poet)
    b=sorted(b.items(), key=lambda x:x[1],reverse=True)
    first_word=word
    stanzas=[]
    verse=''+first_word
    random_number=np.random.randint(VMIN,VMAX)
    for i in range(random_number):
        next_value=''
        for v in b:
            if v[0].split()[1]==first_word:
                next_value=v[0]
        if next_value=='':
            next_value=random.sample(b,1)[0][0]
        first_word=next_value.split()[0]
        verse=verse+' '+first_word
    stanzas.append(verse)
    return stanzas

In [110]:
def Bidirectional_Bigram_Model(dataframe,VMIN,VMAX,words,selected_poet):
    word=random.sample(words,1)[0]
    right_side=right_side_Model(dataframe,selected_poet,VMIN,VMAX,word)
    left_side=left_side_Model(dataframe,selected_poet,VMIN,VMAX,word)
    return right_side,left_side

In [132]:
center_words=get_center_words(data,'ابر احسنی گنوری')
for i in range(stanza_length):
    for j in range(verse_per_stanza):
        a,b=Bidirectional_Bigram_Model(data,3,4,center_words,'ابر احسنی گنوری')
        temp=''
        for k in reversed(b[0].split()[:-1]):
            temp=temp+k+' '
        for k in a[0].split()[1:]:
            temp=temp+k+' '
        print(temp)
    print('\n')

کا اک خزانہ اس زمیں کی 
خوشیاں آپس کا بیر چھوڑیں مل 
بیل یا بھینس بیل یا موتی 
مجھ روشنی مٹی سونا اگل رہی 


اگل آپس کا بیر چھوڑیں مل 
جوتیں بوئیں یہی میری کھیتی ہے 
جوتیں بوئیں یہی میری کھیتی بڑھائیں 
اونٹ گھوڑے پہاڑ سے کیا یہاں 


کیا سے مٹی سونا اگل رہی 
اٹھا بل پہ ساری دنیا سنبھل 
سنبھلیں خوشیاں نہ کیوں ہوئے ہیں 
یا بھینس بیل یا موتی جڑے 


آبرو یاقوت بھی یہی میری کھیتی 
ایسی زمین پا کر ایک مخلوق 
ساری سنبھلیں دنیا سنبھل رہی ہے 
کو نے میرے لیے تیرا گن 




# THE BEAUTIFUL AND POWERFUL END OF THE NOTEBOOK